In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
import torch
from torch.utils.data import Dataset, DataLoader
import torch.nn as nn
import torch.optim as optim
import matplotlib.pyplot as plt

In [2]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/

In [3]:
#!/bin/bash
!kaggle datasets download zalando-research/fashionmnist

Dataset URL: https://www.kaggle.com/datasets/zalando-research/fashionmnist
License(s): other
fashionmnist.zip: Skipping, found more recently modified local copy (use --force to force download)


In [ ]:
import zipfile

zip_ref = zipfile.ZipFile('/content/fashionmnist.zip')  
zip_ref.extractall('/content')  
zip_ref.close()

In [5]:
torch.manual_seed(42)

In [6]:
# Check for GPU
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")

Using device: cuda


In [7]:
train = pd.read_csv('/content/fashion-mnist_train.csv')
test = pd.read_csv('/content/fashion-mnist_test.csv')

In [8]:
x_train = train.iloc[:,1:]
y_train = train.iloc[:,0]
x_test = test.iloc[:,1:]
y_test = test.iloc[:,0]

In [9]:
from torchvision.transforms import transforms
custom_transform = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485,0.456,0.406],std=[0.229,0.224,0.225])
])

In [10]:
from PIL import Image
import numpy as np


In [11]:
class CustomDataset(Dataset):

  def __init__(self,features,labels,transform):
    self.features = features.values
    self.labels = labels
    self.transform = transform

  def __len__(self):
    return len(self.features)

  def __getitem__(self,index):
    image = self.features[index].reshape(28,28)
    image = Image.fromarray(image.astype(np.uint8), mode='L').convert('RGB')


    image = self.transform(image)

    return image,torch.tensor(self.labels[index],dtype=torch.long)

In [12]:
train_dataset = CustomDataset(x_train,y_train,transform=custom_transform)
test_dataset = CustomDataset(x_test,y_test,transform=custom_transform)


In [13]:
train_loader = DataLoader(train_dataset,batch_size=32,shuffle=True)
test_loader = DataLoader(test_dataset,batch_size=32,shuffle=False)

In [14]:
import torchvision.models as models
vgg16 = models.vgg16(pretrained = True)

/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=VGG16_Weights.IMAGENET1K_V1`. You can also use `weights=VGG16_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [15]:
for param in vgg16.features.parameters():
  param.requires_grad=False

In [16]:
vgg16.classifier = nn.Sequential(
    nn.Linear(25088,1024),
    nn.ReLU(),
    nn.Dropout(0.5),
    nn.Linear(1024,512),
    nn.ReLU(),
    nn.Dropout(0.5),
    nn.Linear(512,10),
)

In [17]:
vgg16 = vgg16.to(device)

In [18]:
learning_rate = 0.0001
epochs = 10

In [19]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(vgg16.classifier.parameters(),lr = learning_rate,weight_decay=1e-4)

In [ ]:
for epoch in range(epochs):  
  total_epoch_loss = 0
  correct = 0
  total = 0

  for x, y in train_loader:
    x, y = x.to(device), y.to(device)

    # forward prop
    outputs = vgg16(x)
    # calculate loss
    loss = criterion(outputs, y)
    # back prop
    optimizer.zero_grad()
    loss.backward()

    # update grads
    optimizer.step()

    total_epoch_loss += loss.item()

    # Calculate accuracy
    _, predicted = torch.max(outputs.data, 1)
    total += y.size(0)
    correct += (predicted == y).sum().item()

  avg_loss = total_epoch_loss / len(train_loader)
  accuracy = 100 * correct / total
  print(f"Epoch: {epoch + 1}, Loss: {avg_loss:.4f}, Accuracy: {accuracy:.2f}%")

Epoch: 1, Loss: 0.3468, Accuracy: 87.94%
Epoch: 2, Loss: 0.2140, Accuracy: 92.39%
Epoch: 3, Loss: 0.1703, Accuracy: 93.82%
Epoch: 4, Loss: 0.1395, Accuracy: 94.98%
Epoch: 5, Loss: 0.1160, Accuracy: 95.86%
Epoch: 6, Loss: 0.0967, Accuracy: 96.51%
Epoch: 7, Loss: 0.0835, Accuracy: 96.97%
Epoch: 8, Loss: 0.0747, Accuracy: 97.32%
Epoch: 9, Loss: 0.0664, Accuracy: 97.58%
Epoch: 10, Loss: 0.0600, Accuracy: 97.84%
